In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
main_data = pd.read_csv('FIFA2019.csv', index_col=0)

In [18]:
print(f"Rows : {main_data.shape[0]} x Columns : {main_data.shape[1]}")

np.random.seed(1)
main_data.sample(n=10)

Rows : 18207 x Columns : 88


,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
18070,236705,E. Dahlqvist,19,https://cdn.sofifa.org/players/4/19/236705.png,Sweden,https://cdn.sofifa.org/flags/46.png,50,64,IFK Göteborg,https://cdn.sofifa.org/teams/2/light/319.png,€60K,€1K,1337,Left,1.0,3.0,2.0,Medium/ Medium,Lean,No,LWB,33.0,"Nov 6, 2016",NaN,2021,5'6,154lbs,39+2,39+2,39+2,43+2,41+2,41+2,41+2,43+2,42+2,42+2,42+2,44+2,42+2,42+2,42+2,44+2,48+2,46+2,46+2,46+2,48+2,48+2,48+2,48+2,48+2,48+2,40.0,26.0,37.0,41.0,22.0,44.0,30.0,26.0,36.0,45.0,62.0,57.0,73.0,48.0,84.0,23.0,92.0,64.0,57.0,25.0,53.0,51.0,43.0,36.0,31.0,52.0,40.0,48.0,51.0,8.0,7.0,12.0,13.0,12.0,€113K
5073,233301,R. Nissen Kristensen,20,https://cdn.sofifa.org/players/4/19/233301.png,Denmark,https://cdn.sofifa.org/flags/13.png,70,79,Ajax,https://cdn.sofifa.org/teams/2/light/245.png,€2.4M,€6K,1855,Right,1.0,3.0,3.0,High/ High,Normal,No,RB,2.0,"Jan 23, 2018",NaN,2022,6'1,154lbs,65+2,65+2,65+2,65+2,65+2,65+2,65+2,65+2,64+2,64+2,64+2,67+2,66+2,66+2,66+2,67+2,69+2,68+2,68+2,68+2,69+2,69+2,67+2,67+2,67+2,69+2,66.0,53.0,65.0,67.0,49.0,65.0,68.0,36.0,66.0,67.0,75.0,76.0,66.0,69.0,59.0,73.0,73.0,87.0,74.0,49.0,70.0,67.0,67.0,63.0,45.0,72.0,66.0,66.0,64.0,10.0,9.0,9.0,6.0,10.0,€3.8M
14368,231064,T. Hudson-Wihongi,23,https://cdn.sofifa.org/players/4/19/231064.png,New Zealand,https://cdn.sofifa.org/flags/198.png,61,67,NaN,https://cdn.sofifa.org/flags/198.png,€0,€0,1613,Right,1.0,3.0,2.0,Medium/ Medium,Normal,No,CDM,21.0,NaN,NaN,NaN,6'1,183lbs,53+2,53+2,53+2,56+2,55+2,55+2,55+2,56+2,57+2,57+2,57+2,58+2,58+2,58+2,58+2,58+2,59+2,60+2,60+2,60+2,59+2,59+2,60+2,60+2,60+2,59+2,53.0,45.0,60.0,62.0,27.0,57.0,46.0,37.0,61.0,60.0,70.0,65.0,62.0,56.0,60.0,41.0,62.0,68.0,70.0,48.0,72.0,58.0,50.0,56.0,52.0,65.0,56.0,58.0,57.0,10.0,6.0,6.0,14.0,8.0,NaN
6757,234022,Adrián Maideira,34,https://cdn.sofifa.org/players/4/19/234022.png,Brazil,https://cdn.sofifa.org/flags/54.png,68,68,Vitória,https://cdn.sofifa.org/teams/2/light/1719.png,€450K,€6K,1753,Left,1.0,3.0,3.0,Medium/ Medium,Normal,No,CAM,10.0,"Jan 1, 2018",NaN,2021,5'7,146lbs,67+2,67+2,67+2,68+2,68+2,68+2,68+2,68+2,67+2,67+2,67+2,66+2,61+2,61+2,61+2,66+2,53+2,50+2,50+2,50+2,53+2,51+2,44+2,44+2,44+2,51+2,55.0,67.0,54.0,70.0,66.0,68.0,65.0,62.0,51.0,63.0,80.0,79.0,82.0,70.0,75.0,70.0,58.0,66.0,58.0,72.0,52.0,33.0,71.0,58.0,68.0,59.0,15.0,41.0,38.0,8.0,8.0,9.0,8.0,13.0,€855K
7829,246056,E. Vega,28,https://cdn.sofifa.org/players/4/19/246056.png,Ecuador,https://cdn.sofifa.org/flags/57.png,67,68,NaN,https://cdn.sofifa.org/flags/57.png,€0,€0,1706,Right,1.0,3.0,3.0,Medium/ Medium,Stocky,No,CDM,17.0,NaN,NaN,NaN,5'6,152lbs,52+2,52+2,52+2,56+2,55+2,55+2,55+2,56+2,57+2,57+2,57+2,60+2,62+2,62+2,62+2,60+2,65+2,66+2,66+2,66+2,65+2,65+2,64+2,64+2,64+2,65+2,55.0,19.0,67.0,70.0,12.0,66.0,48.0,59.0,68.0,66.0,68.0,65.0,63.0,64.0,77.0,66.0,66.0,79.0,64.0,41.0,71.0,68.0,38.0,55.0,60.0,54.0,59.0,64.0,62.0,11.0,14.0,9.0,6.0,6.0,NaN
8497,202179,Z. Ismail,24,https://cdn.sofifa.org/players/4/19/202179.png,England,https://cdn.sofifa.org/flags/14.png,67,70,Walsall,https://cdn.sofifa.org/teams/2/light/1803.png,€975K,€3K,1688,Right,1.0,5.0,4.0,High/ Medium,Normal,No,RM,10.0,"Jul 1, 2018",NaN,2019,5'10,165lbs,62+2,62+2,62+2,66+2,64+2,64+2,64+2,66+2,64+2,64+2,64+2,66+2,59+2,59+2,59+2,66+2,53+2,48+2,48+2,48+2,53+2,50+2,41+2,41+2,41+2,50+2,57.0,60.0,45.0

In [19]:
main_data.drop(['ID', 'Photo', 'Flag', 'Club Logo', 'Real Face', 'Jersey Number'], axis=1, inplace=True)

In [20]:
nulls_series             = main_data.isna().sum()         # Get a series counting number of empty values for each column
nonnulls_series          = main_data.notnull().sum()      # Get a series counting number of non empty valuesfor each column 
nulls_percentage         = ((nulls_series * 100)/(nulls_series + nonnulls_series)).astype(float)
column_datatypes         = main_data.dtypes               # Get a series containing data types for each column 

nulls_count_df = pd.concat(
    [nulls_series, nonnulls_series, nulls_percentage, column_datatypes],
    axis=1,
    keys=["Nulls", "Non-Nulls","Nulls %", "Type"], 
    sort=True)
cm = sns.light_palette("red", as_cmap=True)

# nulls_count_df.style.background_gradient(cmap=cm, subset=pd.IndexSlice[:, ['Nulls %']]).format(formatter={('Nulls %'): "{:.2f}%"})

In [21]:
money_cols = []
for col_name in main_data.columns[main_data.dtypes == "object"]:
    if main_data[col_name].str.startswith('€').any():
        money_cols.append(col_name)
        
print(money_cols)

['Value', 'Wage', 'Release Clause']


In [22]:
def income_to_num(income_val):
    if isinstance(income_val, str):
        multiplier = 1
        if income_val.endswith('K'):
            multiplier = 1000
        elif income_val.endswith('M'):
            multiplier = 1000000
        return float(income_val.replace('€', '').replace('K', '').replace('M', '')) * multiplier
    else:
        return np.nan
    
for col_name in money_cols:
    main_data[col_name] = main_data[col_name].apply(income_to_num)
    
main_data[money_cols].head()

,Value,Wage,Release Clause
0,110500000.0,565000.0,226500000.0
1,77000000.0,405000.0,127100000.0
2,118500000.0,290000.0,228100000.0
3,72000000.0,260000.0,138600000.0
4,102000000.0,355000.0,196400000.0


In [23]:
def position_to_num(pos_val):
    if isinstance(pos_val, str):
        return float(pos_val.split('+')[0])
    else:
        return np.nan
    
position_cols = ['LS', 'ST', 'RS', 'LW', 'LF','CF', 'RF', 'RW',
                 'LAM', 'CAM', 'RAM', 'LM','LCM', 'CM', 'RCM',
                'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB',
                'LCB', 'CB', 'RCB', 'RB']

for col_name in position_cols:
    main_data[col_name] = main_data[col_name].apply(position_to_num)
    
# main_data[position_cols].head(200)

In [32]:
def height_to_num(height):
    if isinstance(height, str):
        splt = str(height).split("'")
        return float(splt[0]) * 12 + float(splt[1])
    else:
        return np.nan

def weight_to_num(weight):
    if isinstance(weight, str):
        return float(str(weight).replace('lbs', ''))
    else:
        return np.nan

col_transformations ={
    'Height' : height_to_num,
    'Weight' : weight_to_num
}

for k,v in col_transformations.items():
    main_data[k] = main_data[k].apply(v)
    
#main_data[[*col_transformations]].head(200)

In [25]:
main_data['Joined'] = pd.to_datetime(main_data['Joined'])
main_data['Joined Year'] = main_data['Joined'].dt.year
print(min(main_data['Joined']), min(main_data['Joined']))
main_data['Joined'].head()

1991-06-01 00:00:00 1991-06-01 00:00:00


0   2004-07-01
1   2018-07-10
2   2017-08-03
3   2011-07-01
4   2015-08-30
Name: Joined, dtype: datetime64[ns]

In [26]:
main_data[main_data['Joined'] == min(main_data['Joined'])]

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Joined Year
4741,O. Pérez,45,Mexico,71,71,Pachuca,160000.0,8000.0,1256,Right,2.0,3.0,1.0,Medium/ Medium,Stocky,GK,1991-06-01,NaN,2021,68.0,165.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,14.0,24.0,26.0,13.0,12.0,12.0,20.0,27.0,23.0,60.0,57.0,69.0,72.0,69.0,24.0,85.0,41.0,66.0,18.0,26.0,12.0,12.0,40.0,23.0,62.0,23.0,12.0,11.0,70.0,64.0,65.0,73.0,74.0,272000.0,1991.0


In [27]:
main_data['Work Rate'].head()

0    Medium/ Medium
1         High/ Low
2      High/ Medium
3    Medium/ Medium
4        High/ High
Name: Work Rate, dtype: object

In [28]:
work_rate = main_data['Work Rate'].str.split("/ ",n = 1, expand = True)
work_rate.head()

,0,1
0,Medium,Medium
1,High,Low
2,High,Medium
3,Medium,Medium
4,High,High


In [29]:
main_data.drop(['Work Rate'],axis=1, inplace=True)
main_data['Workrate_Attack'] = work_rate[0]
main_data['Workrate_Defense'] = work_rate[1]
main_data.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Body Type,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Joined Year,Workrate_Attack,Workrate_Defense
0,L. Messi,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,Left,5.0,4.0,4.0,Messi,RF,2004-07-01,NaN,2021,67.0,159.0,88.0,88.0,88.0,92.0,93.0,93.0,93.0,92.0,93.0,93.0,93.0,91.0,84.0,84.0,84.0,91.0,64.0,61.0,61.0,61.0,64.0,59.0,47.0,47.0,47.0,59.0,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0,2004.0,Medium,Medium
1,Cristiano Ronaldo,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,Right,5.0,4.0,5.0,C. Ronaldo,ST,2018-07-10,NaN,2022,74.0,183.0,91.0,91.0,91.0,89.0,90.0,90.0,90.0,89.0,88.0,88.0,88.0,88.0,81.0,81.0,81.0,88.0,65.0,61.0,61.0,61.0,65.0,61.0,53.0,53.0,53.0,61.0,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0,2018.0,High,Low
2,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,Right,5.0,5.0,5.0,Neymar,LW,2017-08-03,NaN,2022,69.0,150.0,84.0,84.0,84.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,88.0,81.0,81.0,81.0,88.0,65.0,60.0,60.0,60.0,65.0,60.0,47.0,47.0,47.0,60.0,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0,2017.0,High,Medium
3,De Gea,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,Right,4.0,3.0,1.0,Lean,GK,2011-07-01,NaN,2020,76.0,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0,2011.0,Medium,Medium
4,K. De Bruyne,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,Right,4.0,5.0,4.0,Normal,RCM,2015-08-30,NaN,2023,71.0,154.0,82.0,82.0,82.0,87.0,87.0,87.0,87.0,87.0,88.0,88.0,88.0,88.0,87.0,87.0,87.0,88.0,77.0,77.0,77.0,77.0,77.0,73.0,66.0,66.0,66.0,73.0,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0,2015.0,High,High


In [30]:
positiontype_to_cols = {
    'Attack'   : ['LS', 'ST', 'RS', 'LF', 'CF', 'RF'],
    'Midfield' : ['LW', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'CDM', 'RDM', 'LDM'],
    'Defense'  : ['LWB', 'RWB', 'LB', 'LCB', 'CB', 'RCB','RB']
}

for pos_type, colvec in positiontype_to_cols.items():
    main_data[pos_type + '_rate'] = round(main_data[colvec].mean(axis=1))
    
main_data.drop(position_cols, axis=1, inplace=True)
print(main_data.shape)

(18207, 61)


In [58]:
data = pd.DataFrame({"Col1": [100,200,300,400], "Col2":[500,600,700,800], "Col3":[900,1000,1100,1200], "Col4":["Nature","Wildlife","Animals","Humans"]})

In [61]:
data["Col4"].str.replace("Nature","Beauty", axis=0,, inplace=False)


SyntaxError: invalid syntax (<ipython-input-61-d13a14c7758d>, line 1)

In [60]:
data

,Col1,Col2,Col3,Col4
0,100,500,900,Nature
1,200,600,1000,Wildlife
2,300,700,1100,Animals
3,400,800,1200,Humans


In [85]:
data = pd.DataFrame({"Col1": [100,200,300,400], "Col2":[500,600,700,800], "Col3":[900,1000,1100,1200], "Col4":["Nature","Wildlife","Animals","Humans"]})
data

,Col1,Col2,Col3,Col4
0,100,500,900,Nature
1,200,600,1000,Wildlife
2,300,700,1100,Animals
3,400,800,1200,Humans


In [86]:
data.mean(axis=1)

0    500.0
1    600.0
2    700.0
3    800.0
dtype: float64

In [87]:
np.mean(data, axis=1)

0    500.0
1    600.0
2    700.0
3    800.0
dtype: float64

In [88]:
data["Col4"].replace("Nature","Beauty", inplace=False)
# data

0      Beauty
1    Wildlife
2     Animals
3      Humans
Name: Col4, dtype: object

In [89]:
data["Col4"].str.replace("Nature","Beauty")

0      Beauty
1    Wildlife
2     Animals
3      Humans
Name: Col4, dtype: object

In [90]:
data

,Col1,Col2,Col3,Col4
0,100,500,900,Nature
1,200,600,1000,Wildlife
2,300,700,1100,Animals
3,400,800,1200,Humans


In [69]:
data["Col5"] = data[["Col1","Col2","Col3"]].sum(axis=1)
data["Col5"]

0    1500
1    1800
2    2100
3    2400
Name: Col5, dtype: int64

In [73]:
data["Col6"] = data["Col3"] - data["Col1"]
data["Col6"]

0    800
1    800
2    800
3    800
Name: Col6, dtype: int64

In [91]:
data.drop(["Col2"],axis=1)

,Col1,Col3,Col4
0,100,900,Nature
1,200,1000,Wildlife
2,300,1100,Animals
3,400,1200,Humans


In [92]:
data

,Col1,Col2,Col3,Col4
0,100,500,900,Nature
1,200,600,1000,Wildlife
2,300,700,1100,Animals
3,400,800,1200,Humans


In [93]:
main_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,18207.0,2.512221e+01,4.669943e+00,16.0,21.0,25.0,28.0,45.0
Overall,18207.0,6.623870e+01,6.908930e+00,46.0,62.0,66.0,71.0,94.0
Potential,18207.0,7.130730e+01,6.136496e+00,48.0,67.0,71.0,75.0,95.0
Value,18207.0,2.410696e+06,5.594933e+06,0.0,300000.0,675000.0,2000000.0,118500000.0
Wage,18207.0,9.731312e+03,2.199929e+04,0.0,1000.0,3000.0,9000.0,565000.0
Special,18207.0,1.597810e+03,2.725860e+02,731.0,1457.0,1635.0,1787.0,2346.0
International Reputation,18159.0,1.113222e+00,3.940308e-01,1.0,1.0,1.0,1.0,5.0
Weak Foot,18159.0,2.947299e+00,6.604563e-01,1.0,3.0,3.0,3.0,5.0
Skill Moves,18159.0,2.361308e+00,7.561641e-01,1.0,2.0,2.0,3.0,5.0
Height,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
